Stratégie de cross validation : Il est important que la cross validation soit bien ficelée
- 1/ sort_value ascending=true sur timestamp
- 2/ Découpage en 5 morceaux
- 3/ Premier CV sur les 2 premiers morceaux dans le temps
- 4/ Second CV sur les 3 premiers morceaux dans le temps
- 5/ Ainsi de suite jusqu'à la fin.